In [1]:
!pip install scipy

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('train.csv')
df

,ID,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,TRAIN_000,4,103,60,33,192,24.0,0.966,33,0
1,TRAIN_001,10,133,68,0,0,27.0,0.245,36,0
2,TRAIN_002,4,112,78,40,0,39.4,0.236,38,0
3,TRAIN_003,1,119,88,41,170,45.3,0.507,26,0
4,TRAIN_004,1,114,66,36,200,38.1,0.289,21,0
...,...,...,...,...,...,...,...,...,...,...
647,TRAIN_647,1,91,64,24,0,29.2,0.192,21,0
648,TRAIN_648,10,122,68,0,0,31.2,0.258,41,0
649,TRAIN_649,8,84,74,31,0,38.3,0.457,39,0
650,TRAIN_650,2,81,72,15,76,30.1,0.547,25,0


In [4]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,652.000000,652.000000,652.000000,652.000000,652.000000,652.000000,652.000000,652.000000,652.000000
mean,3.923313,120.946319,68.895706,20.440184,80.878834,31.947393,0.465133,33.510736,0.349693
std,3.327529,31.992394,19.183843,15.918604,117.748315,7.460975,0.329496,11.824817,0.477239
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.084000,21.000000,0.000000
25%,1.000000,99.000000,64.000000,0.000000,0.000000,27.400000,0.238750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,34.000000,32.000000,0.363000,29.500000,0.000000
75%,6.000000,142.000000,80.000000,32.000000,126.000000,36.425000,0.607750,41.000000,1.000000
max,15.000000,199.000000,114.000000,99.000000,846.000000,57.300000,2.329000,81.000000,1.000000


In [5]:
zero_cols = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[zero_cols] = df[zero_cols].replace(0, pd.NA)
df[zero_cols].isna().sum()

Glucose            4
BloodPressure     30
SkinThickness    195
Insulin          318
BMI                7
dtype: int64

In [6]:
df[zero_cols].isna().mean().sort_values(ascending=False)

Insulin          0.487730
SkinThickness    0.299080
BloodPressure    0.046012
BMI              0.010736
Glucose          0.006135
dtype: float64

In [7]:
from scipy.stats import ttest_ind

In [21]:
df_t = df[["Outcome","Glucose"]].dropna()
df_t

,Outcome,Glucose
0,0,103
1,0,133
2,0,112
3,0,119
4,0,114
...,...,...
647,0,91
648,0,122
649,0,84
650,0,81


In [29]:
g0 = df_t[df_t.Outcome==0]["Glucose"]
g1 = df_t[df_t.Outcome==1]["Glucose"]

In [30]:
stat, p = ttest_ind(g0, g1, equal_var=False)

In [28]:
df_t['Glucose'] = pd.to_numeric(df_t['Glucose']) # Glucose가 object -> 숫자로

In [31]:
stat, p

(np.float64(-13.717679471466164), np.float64(1.885971171556785e-35))

In [32]:
import numpy as np

In [35]:
sd_pooled = np.sqrt((g0.var(ddof=1) + g1.var(ddof=1))/2)

In [36]:
d = (g1.mean() - g0.mean()) / sd_pooled
d

np.float64(1.1555199568312366)

In [41]:
df_t[['Glucose', 'Outcome']].corr()

,Glucose,Outcome
Glucose,1.000000,0.491411
Outcome,0.491411,1.000000


In [42]:
from scipy.stats import spearmanr, pearsonr

In [43]:
pair = df[['BMI', "BloodPressure"]].dropna().copy()

In [48]:
pair = pair.apply(pd.to_numeric) # 데이터 타입을 바꾸는 작업

In [49]:
pair['BMI']

0      24.0
1      27.0
2      39.4
3      45.3
4      38.1
       ... 
647    29.2
648    31.2
649    38.3
650    30.1
651    26.5
Name: BMI, Length: 620, dtype: float64

In [50]:
stat, p = pearsonr(pair['BMI'], pair['BloodPressure'])
stat, p

(np.float64(0.29801020409487106), np.float64(3.501698019719671e-14))

In [51]:
stat2, p2 = spearmanr(pair['BMI'], pair['BloodPressure'])
stat2, p2

(np.float64(0.31620809143695044), np.float64(7.294507587541999e-16))

In [54]:
cut = pd.cut(df_t["Glucose"], bins=[0, 125, 500],
             labels=["정상~전단계","고혈당"], include_lowest=True)

tab = pd.crosstab(cut, df["Outcome"])

In [55]:
tab

Outcome,0,1
Glucose,,
정상~전단계,316,74
고혈당,106,152


In [56]:
from scipy.stats import chi2_contingency


In [57]:
chi2, p, dof, expected= chi2_contingency(tab)
chi2, p, dof, expected

(np.float64(107.3078642284047),
 np.float64(3.811190653879445e-25),
 1,
 array([[253.98148148, 136.01851852],
        [168.01851852,  89.98148148]]))